In [1]:
import lightgbm as lgb
import os
import math
import pandas as pd
import numpy as np
import random
import gc
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  OneHotEncoder

np.random.seed(2019)
random.seed(2019)
pd.set_option('display.max_rows', 4)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 280)
pd.set_option('display.max_colwidth', 150)
data_path = '/data/workspace/kimi/tencent_ads/2020/dataset'
preprocess_path = 'preprocess'

In [2]:
flag ='test'
user_base_statics_df= pd.read_pickle(f'{preprocess_path}/{flag}_user_base_statics.pkl')
user_base_statics_df = user_base_statics_df.reset_index()
user_base_statics_df['click_times_sum_log'] = user_base_statics_df['click_times_sum'].apply(lambda x :math.log(x))
user_base_statics_df['click_times_count_log'] = user_base_statics_df['click_times_count'].apply(lambda x :math.log(x))
user_base_statics_df = user_base_statics_df.astype(float)
print(user_base_statics_df)


          user_id  click_times_sum  click_times_count  ad_id_nunique  product_id_nunique  product_category_nunique  advertiser_id_nunique  industry_nunique  click_times_sum_log  click_times_count_log
0       3000001.0             11.0               11.0           11.0                 7.0                       4.0                   10.0               5.0             2.397895               2.397895
1       3000002.0             65.0               61.0           49.0                24.0                       5.0                   41.0              21.0             4.174387               4.110874
...           ...              ...                ...            ...                 ...                       ...                    ...               ...                  ...                    ...
999998  3999999.0             86.0               80.0           79.0                26.0                       4.0                   63.0              29.0             4.454347               4.382027


In [3]:
def merge_features(train_df,train_file,target_encode=False):
    train_features_df  = pd.read_pickle(f'{preprocess_path}/{train_file}')
    if target_encode:
        train_features_df.columns = [ '_'.join(i) for i in train_features_df.columns.values  ]

    train_df = train_df.merge(train_features_df,on='user_id')
    print(train_df)
    return train_df


In [4]:
test_df = user_base_statics_df
for i in ['creative_id','ad_id', 'product_id','advertiser_id','industry','product_category']:
    print(f'merge {i}...')
    test_df = merge_features(test_df,f'test_user_target_encoder_{i}_age.pkl',True)
    print(test_df)

merge creative_id...
          user_id  click_times_sum  click_times_count  ad_id_nunique  product_id_nunique  ...  creative_id_age8_kfold_mean_std  creative_id_age9_kfold_mean_min  creative_id_age9_kfold_mean_max  creative_id_age9_kfold_mean_mean  creative_id_age9_kfold_mean_std
0       3000001.0             11.0               11.0           11.0                 7.0  ...                         0.005073                              0.0                         0.005254                          0.000568                         0.001583
1       3000002.0             65.0               61.0           49.0                24.0  ...                         0.037389                              0.0                         0.333333                          0.020031                         0.048684
...           ...              ...                ...            ...                 ...  ...                              ...                              ...                              ...       

In [5]:
# train_x = train_df.drop(['age','user_id','gender','advertiser_id_seq','industry_seq','product_id_seq'], axis=1)
# train_y = train_df['age'].astype(int)

# valid_x = valid_df.drop(['age','user_id','gender','advertiser_id_seq','industry_seq','product_id_seq'], axis=1)
# valid_y = valid_df['age'].astype(int)

drop_list = ['user_id','click_times_sum','click_times_count']
test_df = test_df.drop(drop_list, axis=1)





In [6]:

gbm = lgb.Booster(model_file='model/age_target.model')
print('Feature importances:', list(gbm.feature_importance()))

Feature importances: [430, 517, 191, 527, 661, 532, 546, 42, 482, 777, 478, 113, 559, 1158, 538, 160, 548, 941, 539, 148, 413, 817, 513, 114, 416, 762, 447, 166, 502, 978, 538, 97, 471, 915, 492, 53, 426, 822, 466, 39, 360, 686, 422, 18, 392, 536, 402, 25, 475, 635, 491, 160, 515, 1340, 623, 233, 630, 1157, 549, 187, 487, 967, 563, 128, 412, 735, 434, 205, 463, 956, 508, 111, 453, 1012, 466, 64, 430, 904, 425, 30, 405, 632, 424, 26, 361, 579, 373, 351, 430, 499, 396, 484, 491, 480, 376, 385, 468, 424, 305, 365, 407, 401, 317, 377, 449, 439, 420, 344, 442, 433, 366, 409, 414, 515, 330, 400, 324, 352, 301, 271, 345, 350, 325, 250, 461, 463, 396, 764, 480, 747, 502, 1539, 984, 1175, 927, 1140, 876, 865, 651, 828, 1007, 585, 607, 832, 864, 812, 504, 930, 772, 701, 537, 961, 885, 680, 455, 857, 765, 614, 462, 651, 516, 553, 351, 449, 529, 768, 389, 739, 368, 814, 583, 325, 489, 514, 507, 287, 654, 504, 383, 536, 472, 743, 669, 617, 381, 609, 487, 469, 331, 561, 499, 523, 353, 517, 462, 585,

In [7]:
y_pred = gbm.predict(test_df.astype(float))
y_pred_precent = y_pred.copy()
for i in range(len(y_pred)):
        max_value=max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value==y_pred[i][j]:
                y_pred[i][j]=1
            else:
                y_pred[i][j]=0
print(y_pred)             

ret = []
for user_id,age,age_precent in zip(range(1000000),y_pred,y_pred_precent):
    ret.append([int(user_id+3000001),int(age.tolist().index(1) + 1),age_precent])

ret_df = pd.DataFrame(ret,columns=['user_id','predicted_age','lgb_age_precent'])
ret_df.to_pickle("output/lgb_test_output.pkl")


[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
